<a href="https://colab.research.google.com/github/isikaykarakus/MineSweeper/blob/main/Minesweeper_IsikayKarakus_2071938.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Minesweeper Solver Using SAT**

This project is part of the course **Knowledge Representation and Learning 2023-2024** taught by Prof. Roberto Confalonieri and Prof. Luciano Serafini.

Işıkay KARAKUŞ **2071938**

## **Problem Description**

Minesweeper is a classic puzzle game where mines are hidden in a grid of squares. The objective is to uncover all the cells that do not contain mines. The game provides clues in the form of numbers within revealed cells, indicating how many of the adjacent cells contain mines. The challenge is to use these clues to deduce the location of all the mines without triggering any.

This project leverages a SAT (Satisfiability) solver to systematically determine which cells are safe and which contain mines. SAT solvers are powerful tools that can decide the satisfiability of logical propositions, making them well-suited for solving Minesweeper puzzles.


In [1]:
%pip install python-sat
from itertools import combinations
from pysat.solvers import Minisat22



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.4 MB/s eta 0:00:00


## **Code Explanation**

### **Initialization and Encoding**

The code is structured using a class `MinesweeperSolver` that encapsulates the functionality needed to solve the Minesweeper game using a SAT solver.





In [2]:
class MinesweeperSolver:
    """Purpose: Initializes the solver with the game state and calculates the dimensions of the grid.
       Parameters:
       game: A list of strings representing the Minesweeper grid."""

    def __init__(self, game):
        self.game = game
        self.width = len(game[0])
        self.height = len(game)

    def pos_mat(self, r, c):

        """Purpose: Encodes each cell into a unique number for the SAT solver.
           Parameters:
           r: Row of the cell.
           c: Column of the cell.
           Returns: A unique integer representing the cell."""

        return r * (self.width + 2) + c + 1

    def add_border_constraints(self, solver):
        """Adds constraints for border cells (no mines)."""
        for c in range(self.width + 2):
            solver.add_clause([-self.pos_mat(0, c)])
            solver.add_clause([-self.pos_mat(self.height + 1, c)])
        for r in range(self.height + 2):
            solver.add_clause([-self.pos_mat(r, 0)])
            solver.add_clause([-self.pos_mat(r, self.width + 1)])

### Adding Constraints

#The constraints for the SAT solver are added based on the known cells in the game.


    def add_cell_constraints(self, solver):
        """Adds constraints based on the known cells in the game."""
        for r in range(1, self.height + 1):
            for c in range(1, self.width + 1):
                t = self.game[r - 1][c - 1]
                if t in "1234567":
                    solver.add_clause([-self.pos_mat(r, c)])  # No mines in a cell with a number
                    self.add_number_constraints(solver, r, c, int(t))
                elif t == "0":
                    solver.add_clause([-self.pos_mat(r, c)])  # No mine in a cell with a 0
                    self.add_zero_constraints(solver, r, c)
                elif t == "8":
                    solver.add_clause([-self.pos_mat(r, c)])  # No mines in a cell with an 8
                    self.add_eight_constraints(solver, r, c)

### Number, Zero, and Eight Constraints

#Additional methods handle specific constraints for numbered cells, zero cells, and cells showing an eight.

    def add_number_constraints(self, solver, r, c, num):
        """Adds constraints for cells showing a number (1-7)."""
        cells_around = [
            -self.pos_mat(r - 1, c - 1), -self.pos_mat(r - 1, c), -self.pos_mat(r - 1, c + 1),
            -self.pos_mat(r, c - 1), -self.pos_mat(r, c + 1),
            -self.pos_mat(r + 1, c - 1), -self.pos_mat(r + 1, c), -self.pos_mat(r + 1, c + 1)
        ]
        cells_around_neg = [
            self.pos_mat(r - 1, c - 1), self.pos_mat(r - 1, c), self.pos_mat(r - 1, c + 1),
            self.pos_mat(r, c - 1), self.pos_mat(r, c + 1),
            self.pos_mat(r + 1, c - 1), self.pos_mat(r + 1, c), self.pos_mat(r + 1, c + 1)
        ]

        s_most = list(combinations(cells_around, 1 + num))
        s_least = list(combinations(cells_around_neg, 8 - num + 1))

        for x in s_most:
            solver.add_clause(x)
        for y in s_least:
            solver.add_clause(y)

    def add_zero_constraints(self, solver, r, c):
        """Adds constraints for cells showing a zero."""
        cells_around = [
            self.pos_mat(r - 1, c - 1), self.pos_mat(r - 1, c), self.pos_mat(r - 1, c + 1),
            self.pos_mat(r, c - 1), self.pos_mat(r, c + 1),
            self.pos_mat(r + 1, c - 1), self.pos_mat(r + 1, c), self.pos_mat(r + 1, c + 1)
        ]
        for cell in cells_around:
            solver.add_clause([-cell])

    def add_eight_constraints(self, solver, r, c):
        """Adds constraints for cells showing an eight."""
        cells_around = [
            self.pos_mat(r - 1, c - 1), self.pos_mat(r - 1, c), self.pos_mat(r - 1, c + 1),
            self.pos_mat(r, c - 1), self.pos_mat(r, c + 1),
            self.pos_mat(r + 1, c - 1), self.pos_mat(r + 1, c), self.pos_mat(r + 1, c + 1)
        ]
        for cell in cells_around:
            solver.add_clause([cell])

#### Checking a Cell for Mine

    def check_mine(self, row, col, mine):
        """Checks whether a cell contains a mine or is safe."""
        solver = Minisat22()

        self.add_border_constraints(solver)
        self.add_cell_constraints(solver)

        if mine:
            solver.add_clause([self.pos_mat(row, col)])  # Assume mine
            if not solver.solve():
                print(f"row={row} col={col}, -> No mine")
        else:
            solver.add_clause([-self.pos_mat(row, col)])  # Assume no mine
            if not solver.solve():
                print(f"row={row} col={col}, -> MINE")

#### Solve the Minesweeper game

    def solve(self):
        """Solves the Minesweeper game and prints the results for hidden cells."""
        for r in range(self.height):
            for c in range(self.width):
                if self.game[r][c] == "?":
                    self.check_mine(r + 1, c + 1, mine=True)
                    self.check_mine(r + 1, c + 1, mine=False)



## **Example Usage**

This section demonstrates how to use the `MinesweeperSolver` class with an example game state. The class initialized with a predefined Minesweeper grid and then call the `solve` method to analyze the grid and determine the status of each hidden cell.



In [3]:
# Example usage
game = [
    "001??????",
    "001??????",
    "00112????",
    "00001????",
    "000011???",
    "000001???",
    "000012???",
    "12322????",
    "?????????"
]

solver = MinesweeperSolver(game)
solver.solve()


row=1 col=4, -> No mine
row=2 col=4, -> MINE
row=2 col=5, -> No mine
row=2 col=6, -> No mine
row=3 col=6, -> No mine
row=4 col=6, -> MINE
row=4 col=7, -> No mine
row=5 col=7, -> No mine
row=6 col=7, -> No mine
row=7 col=7, -> MINE
row=8 col=6, -> MINE
row=8 col=7, -> No mine
row=9 col=1, -> No mine
row=9 col=2, -> MINE
row=9 col=3, -> MINE
row=9 col=4, -> MINE
row=9 col=5, -> No mine
row=9 col=6, -> No mine
